In [ ]:
#@title #Erstelle eine Character Card v3 inkl. Welt (SillyTavern) { display-mode: "form" }
#@markdown Führe zuerst diesen Code aus um alle Parameter zu initialisieren

#@markdown Im Verlauf dieses Notebooks erstellst du Schritt für Schritt deine CharCard_v3
#@markdown + 1. Schritt: Creator Metadata
#@markdown +2. Schritt: System Prompt & Jailbreak (Optional)
#@markdown +3. Schritt: Persönlichkeit des Charakters (Essentiell!)\
#@markdown   3.1. Schritt: Persönlichkeit des Charakters (Essentiell!)
#@markdown + 4. Schritt: Tiefe der Persönlichkeit (Optional)
#@markdown +5. Schritt: Welt & Komplexität (Optional)\
#@markdown   5.1. Schritt: Schlüsselwörter und Beschreibung

#@markdown + Erfahre, wie viele Token dein Charakter erzeugt (gpt-3.5-turbo)

#@markdown + 6. Schritt: Fertigstellung\
#@markdown    6.1. Schritt: Speichern und Downloaden


import os
import sys
sys.stdout = open(os.devnull, 'w')
!pip install tiktoken
!pip install panel
!pip install jupyter_bokeh


import json
import param
import tiktoken
import panel as pn
from pathlib import Path
from datetime import datetime
from functools import partial


class CreatorMeta(param.Parameterized):
    creator = param.String(default="", doc="Ersteller der Charakterkarte")
    creator_notes = param.String(default="", doc="Notizen des Erstellers")
    character_version = param.String(default="1.0.0", doc="Versionsnummer der Charakterkarte")
    tags = param.String(default="", doc="Beispiel: Elfe, Fantasie, Mittelalter")

class PromptOverrides(param.Parameterized):
    system_prompt = param.String(default="", doc="Individuelle System-Prompt, die Standardanweisungen ersetzt.")
    post_history_instructions = param.String(default="", doc="„Jailbreak“ kann ermöglichen, dass Filter umgangen werden.")

class Personality(param.Parameterized):
    char_name = param.String(default="", doc="Name des Charakters")
    description = param.String(default="", doc="Beschreibung des Charakters")
    personality = param.String(default="", doc="Persönlichkeitsmerkmale des Charakters\nBeispiel: schüchtern + intelligent + freundlich")
    scenario = param.String(default="", doc="Szenario oder Hintergrundgeschichte des Charakters\nVerwende hier KEIN <START>")
    first_mes = param.String(default="", doc="Erste Nachricht oder Begrüßung des Charakters\nVerwende hier KEIN <START>")
    mes_example = param.String(default="", doc="Beispielnachricht des Charakters\nStarte jede Beispielnachricht mit <START>")

class CharExtens_0(param.Parameterized):
    depth_prompt = param.String(default="", doc="Aufforderung zur weiteren Vertiefung des Charakters")

class CharExtens_1(param.Parameterized):
    depth = param.Integer(4, bounds=(1, 6))
    talkativeness = param.Number(0.5, bounds=(0, None), softbounds=(0, 1))

class CharAlt(param.Parameterized):
    alternate_greetings = param.String(default="", doc="Alternative Begrüßungsnachrichten des Charakters")

class World(param.Parameterized):
    world = param.String(default=None, doc="Name der Welt. Beispiel: Eldoria")

class WorldKeys(param.Parameterized):
    keys = param.String(default="", doc="Schlüsselbegriffe oder Kategorien für die Weltbeschreibung\nBeispel: eldoria, wald, magischer wald  ")
    content = param.String(default="", doc="<START>\n{{user}}: Was ist Eldoria?\n{{char}}: Eldoria ist ein uralter, magischer Wald, der voller geheimnisvoller Kreaturen und leuchtender Pflanzen steckt. Er gilt als das Herzstück der Naturmagie und birgt zahlreiche verborgene Pfade und mystische Orte, die nur von den Mutigsten erkundet werden können.")

tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")


def create_widgets(param_instance, max_items_per_row=2, exclude_counters=None, y=100, x=400, only_param=None):
    if exclude_counters is None:
        exclude_counters = []

    widgets = []
    row = []

    for param_name, param_obj in param_instance.param.objects("existing").items():



        if only_param is not None and param_name not in only_param:
            continue

        if isinstance(param_obj, param.String) and param_name not in ['name']:
            label = pn.pane.Markdown(f"### {param_name.replace('_', ' ').capitalize()}")
            placeholder_text = param_obj.doc if param_obj.doc else f"Enter your {param_name} here..."
            text_area = pn.widgets.TextAreaInput(
                value=getattr(param_instance, param_name),
                placeholder=placeholder_text,
                height=y,
                width=x,
                max_length=10000
            )

            if param_name not in exclude_counters:

                char_counter = pn.pane.Markdown("**0 characters ~ 0 Tokens by Tiktoken**")

                def update_char_count(event, char_counter, param_name = param_name):
                    text = event.new
                    setattr(param_instance, param_name, text)
                    char_count = len(text)
                    token_count = len(tokenizer.encode(text))
                    char_counter.object = f"**{char_count} characters ~ {token_count} Tokens by Tiktoken**"

                text_area.param.watch(partial(update_char_count, char_counter=char_counter), 'value')

                row.append(pn.Column(label, text_area, char_counter))
            else:
                def update_value_no_counter(event, pn_name=param_name):
                    setattr(param_instance, pn_name, event.new)

                text_area.param.watch(partial(update_value_no_counter, pn_name=param_name), 'value')

                row.append(pn.Column(label, text_area))

            if len(row) == max_items_per_row:
                widgets.append(pn.Row(*row))
                row = []

    if row:
        widgets.append(pn.Row(*row))

    return widgets


def read_param_values(param_instance, only_params=None):
    """
    Liest alle Parameterwerte aus dem param_instance aus und gibt sie als Dictionary zurück.
    Falls only_params angegeben ist (Liste von Strings), werden nur diese Parameter ausgelesen.
    """
    values_dict = {}

    for param_name, param_obj in param_instance.param.objects("existing").items():
        if isinstance(param_obj, param.String) and param_name not in ['name']:
            if only_params is not None and param_name not in only_params:
                    continue

        values_dict[param_name] = getattr(param_instance, param_name)

    return values_dict

pn.extension(
    theme='dark',
    raw_css=["textarea { resize: none; }"]
)

button_0 = pn.widgets.Button(name='Eingabe bestätigen')
def on_button(event):
    print('Eingabe wurde bestätigt')

button_0.on_click(on_button)

# Instanz der Klasse
creator_meta = CreatorMeta()
jailbreak = PromptOverrides()
personality = Personality()
char_extens = CharExtens_0()
char_extens_1 = CharExtens_1()
char_alt = CharAlt()
world_name = World()


# Widgets für alle Parameter der Klasse erstellen
widgets_0 = create_widgets(creator_meta, y=35, exclude_counters=['creator', 'creator_notes', 'character_version', 'tags'])
widgets_1 = create_widgets(jailbreak, x=500)
widgets_2 = create_widgets(personality, y = 35, only_param=['char_name'])
widgets_3 = create_widgets(personality, x=600, y=300, only_param=['description', 'personality'])
widgets_3_1 = create_widgets(personality, x=600, y=300, only_param=['scenario', 'first_mes', 'mes_example'])
widgets_4 = create_widgets(char_extens, x=600, y=300, only_param=['depth_prompt'])
widgets_5 = create_widgets(char_alt, x=600, y=300)
widgets_6 = create_widgets(world_name, y = 35, exclude_counters=['world'])

sys.stdout = sys.__stdout__
pn.pane.Markdown("Die Parameter sind nun initialisiert. Du kannst jetzt mit Schritt 1 fortfahren.")


In [ ]:
#@title 1. Schritt: Creator Metadata { display-mode: "form" }
#@markdown In folgendem Abschnitt kannst du Informationen über deine CharCard angeben\
#@markdown Alle Eingaben in diesem Bereich generieren keine Tokens
pn.Column(
    pn.pane.Markdown("## Meta Daten deiner CharCard"),
    *widgets_0,
    button_0
)


In [ ]:
#@title 2. Schritt: System Prompt & Jailbreak (Optional) { display-mode: "form" }
#@markdown Hier hast du die Möglichkeit, einen "Jailbreak" auszuführen (nicht empfohlen)


pn.Column(
    pn.pane.Markdown("## CharCard"),
    *widgets_1,
    button_0
)

In [ ]:
#@title 3. Schritt: Persönlichkeit des Charakters (Essentiell!) { display-mode: "form" }
#@markdown In diesem Schritt erstellst du das Persönlichkeitsprofil deines Charakters

pn.Column(
    pn.pane.Markdown(
        "# Persönlichkeit des Charakters\n"
        ),
    *widgets_2,
    *widgets_3,
    button_0,
    height_policy="fit"
    )

In [ ]:
#@title 3.1. Schritt: Persönlichkeit des Charakters (Essentiell!) { display-mode: "form" }
#@markdown  Bei Beschreibungen von Handlungen verwendest du " \*nonverbale Aktionen* "\
#@markdown Einige dieser Felder benötigen eine bestimmte Art der Formatierung\
#@markdown Formatierung:
#@markdown + Verwende {{char}} anstelle des Charakternamens.
#@markdown + Verwende {{user}} anstelle des Benutzernamens.

#@markdown Beispiel:
#@markdown ```
#@markdown <START>
#@markdown {{user}}: Hallo Aqua, wie läuft's mit deinen Abenteuern?
#@markdown {{char}}: *lacht laut* Oh, meine Abenteuer? Meistens enden sie in Chaos, aber hey, das macht doch den Spaß aus, oder?
#@markdown ```

pn.Column(
    pn.pane.Markdown(
        "# Charakter Kontext\n"
        ),
    *widgets_3_1,
    button_0,
    )


In [ ]:
#@title 4. Schritt: Tiefe der Persönlichkeit (Optional) { display-mode: "form" }
#@markdown Mit der Tiefe der Persönlichkeit wird dein Charakter konsequenter seine Persönlichkeit beibehalten
#@markdown * Weniger Freiheit in der Charakternwicklung
#@markdown * Ihr könnt eine alternative Begrüßungsnachricht hinzufügen
pn.Column(
    pn.pane.Markdown("## Ersteller\n"),
    pn.Row(
        *widgets_4,
        *widgets_5
    ),
    pn.Param(char_extens_1.param, show_name=False),
    button_0
)

In [ ]:
#@title 5. Schritt: Welt & Komplexität (Optional) { display-mode: "form" }
#@markdown Hier bestimmt ihr den Namen der Welt, in der euer Charakter lebt.
#@markdown * Bestimmt, wie komplex eure Welt sein soll\
#@markdown Nur empfohlen für Modelle, die eine große Kontextaufnahme haben\
#@markdown (Meine Empfehlung: Claude 3.5 Haiku)
class ValueInfo(param.Parameterized):
    value_infos = param.Integer(4, bounds=(1, 6))

value_info = ValueInfo()

pn.Column(
    pn.pane.Markdown(
        "## Anzahl der Welteninfos\n"
        "Achtung! Hier sind viele Informationen notwendig\n"
        "Wenn deine Welt komplex werden soll empfehle ich 4 Welteninformationen"
        ),
    pn.Param(value_info.param, show_name=False),
    *widgets_6,
    button_0
    )

In [ ]:
#@title 5.1. Schritt: Schlüsselwörter und Beschreibung { display-mode: "form" }
#@markdown Achtung: *Nur ausführen, wenn du Schritt 5 ausgeführt hast*

value = value_info.value_infos
name = world_name.world
world_keys_list = []

if name is not None and not len(name) == 0:
    def world_value(value, name):

        world_keys = WorldKeys()
        world_keys_list.append(world_keys)

        widgets_7 = create_widgets(world_keys, y = 60, only_param=['keys'])
        widgets_8 = create_widgets(world_keys, x=600, y=300, only_param=['content'])

        return pn.Column(
            pn.pane.Markdown(f"## {value+1}. Beschreibung von {name}"),
            pn.Column(
                *widgets_7,
                *widgets_8
            )
        )

    panels = []
    for i in range(value):
        panels.append(world_value(i, name))


else:
    panels = pn.pane.Markdown('Du hast keinen Namen für eine Welt angegeben!')

pn.Column(*panels, button_0)


In [ ]:
#@title #####Debug (Diesen Abschnitt kannst du ignorieren) { display-mode: "form" }
button = pn.widgets.Button(name='Parameter auslesen')

def on_button_click(event):
    button.disabled = True
    all = [
        read_param_values(creator_meta),
        read_param_values(jailbreak),
        read_param_values(personality),
        read_param_values(char_extens),
        read_param_values(char_extens_1),
        read_param_values(char_alt),
        read_param_values(world_name),
    ]
    if 'world_keys_list' in globals():
        for world_keys in world_keys_list:
            all.append(read_param_values(world_keys))

    print(all)

button.on_click(on_button_click)

pn.Column(button)

In [ ]:
#@title Erfahre, wie viele Token dein Charakter erzeugt (gpt-3.5-turbo) { display-mode: "form" }
#@markdown 4096 Token sind empfohlen, aber einige Modelle (Claude) umfassen einen Input von bis zu 100k Token.
#@markdown - Nach eigener Erfahrung ist weniger mehr.\
#@markdown (Denke daran, dass die Erinnerung auch Token benötigt.)\
#@markdown Grobes Beispiel: Gesamt 8192 Token - Persönlichkeit 2048 Token = Erinnerung 6144 Token.\
#@markdown Die Gesamtaufnahme von Token hängt von eurem Modell ab und davon, was ihr eingestellt habt.

# Funktion zur Berechnung der gesamten Token-Anzahl eines Parameters
def total_token_count(param_instance, encoding):
    total = 0
    for param_name, param_obj in param_instance.param.objects("existing").items():
        if isinstance(param_obj, param.String) and param_name not in ['name']:
            wert = getattr(param_instance, param_name)
            if wert:
                tokens = len(encoding.encode(wert))
                total += tokens
    return total

encoding = tiktoken.encoding_for_model('gpt-3.5-turbo')

gesamt_tokens = (
    total_token_count(personality, encoding) +
    total_token_count(jailbreak, encoding) +
    total_token_count(char_extens, encoding) +
    total_token_count(char_extens_1, encoding) +
    total_token_count(char_alt, encoding) +
    total_token_count(world_name, encoding)
)

if gesamt_tokens >= 4096:
    color = "yellow"
else:
    color = "green"

markdown_text = f"# Gesamt: <span style='color:{color};'>Tokens {gesamt_tokens}</span>"

pn.pane.Markdown(markdown_text).servable()

In [ ]:
#@title 6. Schritt: Fertigstellung { display-mode: "form" }
#@markdown Hier werden deine Eingaben in ein .json Format umgewandelt\
#@markdown Starte den Code und speichere ihn dann in Schritt 6.1

all_para = [
    read_param_values(creator_meta),
    read_param_values(jailbreak),
    read_param_values(personality),
    read_param_values(char_extens),
    read_param_values(char_extens_1),
    read_param_values(char_alt),
    read_param_values(world_name)
    ]

world_para = []
if 'world_keys_list' in globals():
    for world_keys in world_keys_list:
        world_para.append(read_param_values(world_keys))

day = datetime.now().strftime("%Y-%m-%d")
time = datetime.now().strftime("%Hh %Mm %Ss")
# Initialisieren der JSON-Struktur
json_data = {
    "name": "",
    "description": "",
    "personality": "",
    "first_mes": "",
    "avatar": "none",
    "chat": "",
    "mes_example": "",
    "scenario": "",
    "create_date": f"{day} @{time}",
    "talkativeness": "",
    "fav": False,
    "creatorcomment": "",
    "spec": "chara_card_v3",
    "spec_version": "3.0",
    "data": {
        "name": "",
        "description": "",
        "personality": "",
        "scenario": "",
        "first_mes": "",
        "mes_example": "",
        "creator_notes": "",
        "system_prompt": "",
        "post_history_instructions": "",
        "tags": [],
        "creator": "",
        "character_version": "",
        "alternate_greetings": [],
        "extensions": {
            "talkativeness": "0.5",
            "fav": False,
            "world": "",
            "depth_prompt": {
                "prompt": "",
                "depth": 4,
                "role": "system"
            }
        },
        "character_book": {
            "entries": [],
            "name": ""
        },
        "group_only_greetings": []
    },
    "tags": []
}


def assign_parameters(params, json_struct):
    for param in params:

        name = param.get('name')

        if name.startswith('CreatorMeta'):

            json_struct['creatorcomment'] = param.get('creator_notes', '')
            json_struct['data']['creator'] = param.get('creator', '')
            json_struct['data']['creator_notes'] = param.get('creator_notes', '')
            json_struct['data']['character_version'] = param.get('character_version', '')

            tags = param.get('tags', '')
            if isinstance(tags, list):
                json_struct['data']['tags'] = tags
                json_struct['tags'] = tags
            elif isinstance(tags, str):
                json_struct['data']['tags'] = [tags]
                json_struct['tags'] = tags

        elif name.startswith('PromptOverrides'):
            json_struct['data']['system_prompt'] = param.get('system_prompt', '')
            json_struct['data']['post_history_instructions'] = param.get('post_history_instructions', '')

        elif name.startswith('Personality'):
            json_struct['name'] = param.get('char_name', '')
            json_struct['description'] = param.get('description', '')
            json_struct['personality'] = param.get('personality', '')
            json_struct['scenario'] = param.get('scenario', '')
            json_struct['first_mes'] = param.get('first_mes', '')
            json_struct['mes_example'] = param.get('mes_example', '')

            json_struct['data']['name'] = param.get('char_name', '')
            json_struct['data']['description'] = param.get('description', '')
            json_struct['data']['personality'] = param.get('personality', '')
            json_struct['data']['scenario'] = param.get('scenario', '')
            json_struct['data']['first_mes'] = param.get('first_mes', '')
            json_struct['data']['mes_example'] = param.get('mes_example', '')

        elif name.startswith('CharExtens'):
            for key, value in param.items():
                if key == 'name':
                    continue
                if key == 'depth_prompt' and isinstance(value, str):
                    json_struct['data']['extensions']['depth_prompt']['prompt'] = value
                elif key == 'depth':
                    json_struct['data']['extensions']['depth'] = value
                elif key == 'talkativeness':
                    json_struct['talkativeness'] = f"{value}"
                    json_struct['data']['extensions']['talkativeness'] = f"{value}"

        elif name.startswith('World'):
            for key, value in param.items():
                if key == 'name':
                    continue
                elif key == 'world':
                    json_struct['data']['extensions']['world'] = value
                    json_struct['data']['character_book']['name'] = value

        elif name.startswith('CharAlt'):
            alternate_greetings = param.get('alternate_greetings', [])
            if isinstance(alternate_greetings, str):
                json_struct['data']['alternate_greetings'].append(alternate_greetings)
            elif isinstance(alternate_greetings, list):
                json_struct['data']['alternate_greetings'].extend(alternate_greetings)

def add_world_entries(world_keys, json_struct):
    id = 0
    for world_key in world_keys:
        key = world_key.get('keys')
        content = world_key.get('content')

        entry = {
            "id": id,
            "keys": [key],
            "secondary_keys": [],
            "comment": "",
            "content": content,
            "constant": False,
            "selective": True,
            "insertion_order": 100,
            "enabled": True,
            "position": "before_char",
            "use_regex": True,
            "extensions": {
                "position": 0,
                "exclude_recursion": False,
                "display_index": 0,
                "probability": 0,
                "useProbability": True,
                "depth": 4,
                "selectiveLogic": 0,
                "group": "",
                "group_override": False,
                "group_weight": 100,
                "prevent_recursion": False,
                "delay_until_recursion": False,
                "scan_depth": None,
                "match_whole_words": None,
                "use_group_scoring": False,
                "case_sensitive": None,
                "automation_id": "",
                "role": 0,
                "vectorized": False,
                "sticky": 0,
                "cooldown": 0,
                "delay": 0
            }
        }
        json_struct['data']['character_book']['entries'].append(entry)

        id+=1

assign_parameters(all_para, json_data)

if 'world_keys_list' in globals():
    add_world_entries(world_para, json_data)

pn.pane.Markdown("Du kannst deine CharCard jetzt speichern und herunterladen")


In [ ]:
#@title #####Debug (Diesen Abschnitt kannst du ignorieren) { display-mode: "form" }
pn.pane.Markdown(json.dumps(json_data, indent=4, ensure_ascii=False))

In [ ]:
#@title 6.1. Schritt: Speichern und Downloaden { display-mode: "form" }
#@markdown Deine CharCard wird in dem Ordner CharCard gespeichert und heruntergeladen.

from pathlib import Path
from google.colab import files

base_name = personality.char_name if personality.char_name else "CharCard"
base_filename = f"{base_name}.json"
directory = Path('CharCard')
filename = directory / base_filename

directory.mkdir(parents=True, exist_ok=True)

def get_unique_filename(filepath):
    counter = 1
    new_filepath = filepath
    while new_filepath.exists():
        new_filename = f"{filepath.stem}_{counter}{filepath.suffix}"
        new_filepath = filepath.parent / new_filename
        counter += 1
    return new_filepath

unique_filename = get_unique_filename(filename)

with unique_filename.open('w', encoding='utf-8') as file:
    json.dump(json_data, file, indent=4, ensure_ascii=False)

files.download(str(unique_filename))
